# # Investigating San Francisco food health using SQL (sqlite database)

The fictional scenario is a prospective new restaurant in San Francisco, wanting to find a good location. Criteria for our imaginary owner are:
 - staying away from big restaurant owners who own multiple restaurants.
 - picking an area that is trending (has a lot of restaurants).

Using a relational dataset about food health investigations carried out in San Francisco and their outcomes, this notebook goes through a series of SQL queries to answer these business intelligence questions.

# Setup
To install using conda:
```
conda create -n sfrestaurants-sql python=3 jupyter numpy pandas matplotlib seaborn ipython-sql
```
```
conda activate sfrestaurants-sql
```
Then, if not installed previously, 
```
pip install "ipython-cypher==0.2.6"
```

In [1]:
import pandas as pd

In [2]:
%load_ext sql
%sql sqlite:///data/sfscores.sqlite

'Connected: @data/sfscores.sqlite'

## Some basic Exploratory Data Analysis

### Find out sqlite db structure

In [3]:
%%sql 
SELECT name, sql FROM sqlite_master
WHERE type='table'
ORDER BY name;

 * sqlite:///data/sfscores.sqlite
Done.


name,sql
businesses,"CREATE TABLE businesses ( business_id INTEGER NOT NULL, name VARCHAR(64), address VARCHAR(50), city VARCHAR(23), postal_code VARCHAR(9), latitude FLOAT, longitude FLOAT, phone_number BIGINT, ""TaxCode"" VARCHAR(4), business_certificate INTEGER, application_date DATE, owner_name VARCHAR(99), owner_address VARCHAR(74), owner_city VARCHAR(22), owner_state VARCHAR(14), owner_zip VARCHAR(15))"
inspections,"CREATE TABLE inspections (business_id TEXT NOT NULL, Score INTEGER, date INTEGER NOT NULL, type VARCHAR (33) NOT NULL)"
violations,"CREATE TABLE violations (business_id TEXT NOT NULL, date INTEGER NOT NULL, ViolationTypeID TEXT NOT NULL, risk_category TEXT NOT NULL, description TEXT NOT NULL)"


Another way to programmatically find out db structure and list each column individually (note the type etc are also available but I only show table name and column name)

In [4]:
%%sql 
SELECT 
  m.name as table_name, 
  p.name as column_name
FROM 
  sqlite_master AS m
JOIN 
  pragma_table_info(m.name) AS p
ORDER BY 
  m.name, p.cid

 * sqlite:///data/sfscores.sqlite
Done.


table_name,column_name
businesses,business_id
businesses,name
businesses,address
businesses,city
businesses,postal_code
businesses,latitude
businesses,longitude
businesses,phone_number
businesses,TaxCode
businesses,business_certificate


### Show some example data from each table

In [4]:
%sql SELECT * FROM businesses LIMIT 2

 * sqlite:///data/sfscores.sqlite
Done.


business_id,name,address,city,postal_code,latitude,longitude,phone_number,TaxCode,business_certificate,application_date,owner_name,owner_address,owner_city,owner_state,owner_zip
10,Tiramisu Kitchen,033 Belden Pl,San Francisco,94104,37.791116,-122.403816,None,H24,779059,None,Tiramisu LLC,33 Belden St,San Francisco,CA,94104
19,Nrgize Lifestyle Cafe,"1200 Van Ness Ave, 3rd Floor",San Francisco,94109,37.786848,-122.421547,None,H24,None,None,24 Hour Fitness Inc,"1200 Van Ness Ave, 3rd Floor",San Francisco,CA,94109


In [5]:
%sql SELECT * FROM inspections LIMIT 2

 * sqlite:///data/sfscores.sqlite
Done.


business_id,Score,date,type
10,None,20160707,Complaint Reinspection/Followup
10,82,20160503,Routine - Unscheduled


In [6]:
%sql SELECT * FROM violations LIMIT 2

 * sqlite:///data/sfscores.sqlite
Done.


business_id,date,ViolationTypeID,risk_category,description
10,20140729,103129,Moderate Risk,Insufficient hot water or running water
10,20140729,103144,Low Risk,Unapproved or unmaintained equipment or utensils


### how dirty is the data?
I added this section after looking at which fields we are interested in from the queries posed below, e.g. business names, postcodes, city name, etc.

In [7]:
%sql SELECT distinct(postal_code) from businesses

 * sqlite:///data/sfscores.sqlite
Done.


postal_code
94104
94109
94133
94110
94122
94103
94115
94131
94111
94117


In [9]:
%sql SELECT count(postal_code) from businesses WHERE lower(postal_code) = 'ca'

 * sqlite:///data/sfscores.sqlite
Done.


count(postal_code)
5


In [10]:
%sql SELECT * from businesses WHERE lower(postal_code) = 'ca'

 * sqlite:///data/sfscores.sqlite
Done.


business_id,name,address,city,postal_code,latitude,longitude,phone_number,TaxCode,business_certificate,application_date,owner_name,owner_address,owner_city,owner_state,owner_zip
64540,Leo's Hot Dogs,2301 Mission St,S.F.,CA,37.760054,-122.419166,14155246434,H78,421849,2010-10-19,Cesar Ascarrunz,91 Miguel St.,SF,CA,94131
64738,Japacurry,Public,SF,CA,37.777122,-122.419639,None,H79,None,2010-11-16,Mutsuo Hamada,1819 Market St.,San Francisco,CA,94110
81376,Trilogi SF,425 Barneveld Ave,San Francisco,CA,None,None,14155313475,H74,481223,2014-11-11,Vivien Li and Tessa Phosrithong,1718 28th Ave,San Francisco,CA,94122
88139,Tacolicious,2250 Chestnut St,San Francisco,Ca,None,None,14155646077,H26,403812,2016-06-21,Laiola LLC,2250 Chestnut Street,San Francisco,ca,94123
88160,Spice Affairs,1028 Market St,San Francisco,Ca,None,None,14150617277,H24,462426,2016-06-22,Arjun Singh,384 Heather Dr,San Pablo,Ca,94806


In [11]:
%sql SELECT distinct(city) from businesses

 * sqlite:///data/sfscores.sqlite
Done.


city
San Francisco
S.F.
SAN FRANCISCO
SF
Sf
SAN FRANCICSO
Novato
SO. SAN FRANCISCO
FOSTER CITY
DALY CITY


# Queries

## Pt1: Essentials

**1. Write a SQL query that finds the number of business ids in the businesses table**

In [12]:
%sql SELECT COUNT(business_id) FROM businesses

 * sqlite:///data/sfscores.sqlite
Done.


COUNT(business_id)
7634


**2. Write a SQL query that finds out how many unique business names are registered with San Francisco Food health investigation organization and name the column as unique restaurant name count.**

In [13]:
%%sql
SELECT
    COUNT(DISTINCT(name)) as "unique resturant name count"
FROM
    businesses
-- the question assumes all data are reg with San Francisco

 * sqlite:///data/sfscores.sqlite
Done.


unique resturant name count
7059


**3. Write a SQL query that finds out what is the earliest and latest date a health investigation is recorded in this database.**

In [14]:
%sql SELECT MIN(date), MAX(date) FROM inspections

 * sqlite:///data/sfscores.sqlite
Done.


MIN(date),MAX(date)
20131007,20161215


**4. How many businesses are there in San Francisco where their owners live in the same area (postal code/ zip code) as the business is located?**

In [15]:
%sql SELECT COUNT(business_id) from businesses WHERE postal_code = owner_zip

 * sqlite:///data/sfscores.sqlite
Done.


COUNT(business_id)
4059


**5. Out of those businesses, how many of them has a registered business certificate?**

In [16]:
%sql SELECT COUNT(business_id) from businesses WHERE (city = owner_city) AND (business_certificate <> 'none')

 * sqlite:///data/sfscores.sqlite
Done.


COUNT(business_id)
2718


## Pt2: Groupby

**6. Find out the distribution of the risk exposure of all the violations reported in the database. The first column of the result should 'risk_category' and the second column the count.**

In [17]:
%%sql 
SELECT
    risk_category, COUNT(business_id) as "count"
FROM
    violations
GROUP BY
    risk_category


 * sqlite:///data/sfscores.sqlite
Done.


risk_category,count
High Risk,5608
Low Risk,20996
Moderate Risk,14131


**7. Find out the distribution of the risk exposure of all the violations reported in the database that are *water related*. Sort them by frequency (count) from high to low.**

In [18]:
%%sql 
SELECT
    risk_category, COUNT(business_id)
FROM
    violations
WHERE
    description LIKE '%water%'
GROUP BY
    risk_category
ORDER BY 
    risk_category = 'Low Risk' DESC,
    risk_category = 'Moderate Risk' DESC,
    risk_category = 'High Risk' DESC


 * sqlite:///data/sfscores.sqlite
Done.


risk_category,COUNT(business_id)
Moderate Risk,630
High Risk,237


**8. What types of inspections does the authorities conduct and how often do they occur in general.
    Calculate the distribution of different types of inspections with their frequency (type, frequency)
    based on inspections records. Sort them in ascending order based on frequency.**

In [21]:
%%sql 
SELECT
    type, COUNT(business_id) as frequency
FROM
    inspections
GROUP BY
    type
ORDER BY 
    frequency

 * sqlite:///data/sfscores.sqlite
Done.


type,frequency
Multi-agency Investigation,1
Administrative or Document Review,3
Special Event,7
Routine - Scheduled,74
Foodborne Illness Investigation,80
Complaint Reinspection/Followup,138
Structural Inspection,170
Non-inspection site visit,814
Complaint,1625
New Construction,1737


**9. What is the average score given to restaurants based on the type of inspection?
    Based on the results, identify the types of inspections that are not scored (NULL)
    and remove those categories from the resultset. The 'average_score' should be rounded
    to one decimal. Sort the results in ascending order based on the average score. Hint: use the function `ROUND(score, 1)`**

In [22]:
%%sql 
SELECT
    type, ROUND(AVG(score), 1) as "average score"
FROM
    inspections
WHERE
   score <> 'None'
GROUP BY
    type
ORDER BY 
    "average score"

 * sqlite:///data/sfscores.sqlite
Done.


type,average score
Reinspection/Followup,85.0
Routine - Unscheduled,91.2
New Ownership,100.0


**10. Find the restaurant owners (owner_name) that own one or multiple restaurants in the city
    with the number of restaurants (num_restaurants) they own.
    Find the first top 10 owners ordered by descending order using the number of restaurants.**

In [23]:
%%sql 
SELECT
    owner_name, COUNT(business_id) as "num_restaurants"
FROM
    businesses
GROUP BY
    owner_name
ORDER BY
    num_restaurants DESC
LIMIT 10

 * sqlite:///data/sfscores.sqlite
Done.


owner_name,num_restaurants
VOLUME SERVICES AMERICA,94
"Department of Children, Youth and their Families",63
SFUSD,44
"Volume Services, Inc.",41
Children's Empowerment Inc.,37
SAN FRANCISCO UNIFIED SCHOOL,34
WALGREEN CO.,17
SMG Food and Beverage LLC,15
Walgreen Co,15
Walgreen Co.,14


## Pt3: Subqueries and joins

**11. From the businesses table, select the top 10 most popular postal_code. They should be filtered to only count the restaurants owned by people/entities that own 5 or more restaurants. The final result should return a row (postal_code, count) for each 10 selection and be sorted by descending order to get the most relevant zip codes**

In [24]:
%%sql
SELECT COUNT(DISTINCT(postal_code)) from businesses

 * sqlite:///data/sfscores.sqlite
Done.


COUNT(DISTINCT(postal_code))
52


In [25]:
%%sql
SELECT
    b1.postal_code, COUNT(b1.business_id) as "count"
FROM
    businesses as b1
WHERE
    b1.owner_name IN
(
SELECT
    b2.owner_name
FROM
    businesses as b2
GROUP BY
    b2.owner_name
HAVING COUNT(b2.business_id) > 4 
)
GROUP BY 
    b1.postal_code
ORDER BY
    count DESC
LIMIT 10


 * sqlite:///data/sfscores.sqlite
Done.


postal_code,count
94107,179
94103,68
94102,56
94110,32
94112,30
94104,28
94124,27
94105,23
94133,22
94111,20


**12. First let's get an idea about the inspection score our competition has.
    Based on multiple inspections, find out the minimum Score (as "min_score"),
    average Score (as "avg_score") and maximum Score (as "max_score") for all restaurant in post code "94103".
    The average score should be rounded to one decimal.**

In [26]:
%%sql 
SELECT
    min(score) as "min_score"
    , round(avg(score), 1) as "avg_score"
    , max(score) as "max_score"
FROM
    inspections
WHERE
    business_id IN 
    (select business_id from businesses where postal_code = '94103')
AND
    score <> 'None'

 * sqlite:///data/sfscores.sqlite
Done.


min_score,avg_score,max_score
54,90.2,100


In [27]:
%%sql
select (count(score)) 
from inspections
group by business_id

 * sqlite:///data/sfscores.sqlite
Done.


(count(score))
3
3
4
4
3
5
3
4
3
1


In [28]:
%sql select count(business_id) from businesses where postal_code = '94103'

 * sqlite:///data/sfscores.sqlite
Done.


count(business_id)
658


**13. Now lets get more serious, and look at how many times restaurants in Market street
    (postal_code: 94103) has committed health violations and group them based on their risk category.
    The output should be (risk_category, count as frequency) and sorted in descending order by frequency**

This is the answer using an inner join:

In [29]:
%%sql
SELECT
    risk_category, COUNT(v.business_id) as "frequency"
FROM
    violations as v
INNER JOIN
    businesses as b
ON
    v.business_id = b.business_id 
WHERE
    (postal_code = '94103')
GROUP BY
    risk_category
ORDER BY
    frequency DESC


 * sqlite:///data/sfscores.sqlite
Done.


risk_category,frequency
Low Risk,1752
Moderate Risk,1231
High Risk,520


This is the answer using a subquery. It produces the same answer for 
```WHERE (postal_code = '94103') AND (LOWER(address) LIKE #%market s%')```
but **different** numbers when just querying
```WHERE (postal_code = '94103')```.

This is because the data is dirty! There are duplicate records. See further below.

In [31]:
%%sql
SELECT
    risk_category, COUNT(v.business_id) as "frequency"
FROM
    violations as v
WHERE
    v.business_id
IN 
(SELECT b.business_id from businesses as b
WHERE
     (postal_code = '94103'))
GROUP BY
    risk_category
ORDER BY
    frequency DESC

 * sqlite:///data/sfscores.sqlite
Done.


risk_category,frequency
Low Risk,1746
Moderate Risk,1229
High Risk,520


## A bit of investigation into diff between inner join and sub-query

The end result being: The difference is because there are duplicate business records (businesses should be unique), and inner join picks up more of them. Sometimes results like these mean your query needs re-assessing, but sometimes it means your data need more cleaning/quality assessment.

how many businesses have "market" in their address?

In [33]:
%sql select count(business_id) from businesses where (lower(address) LIKE '%market%')

 * sqlite:///data/sfscores.sqlite
Done.


count(business_id)
300


How many business have postcode 94103?

In [34]:
%sql select count(business_id) from businesses where postal_code = '94103'

 * sqlite:///data/sfscores.sqlite
Done.


count(business_id)
658


How many businesses have both "market" in their address and postcode 94103?

In [35]:
%sql select count(business_id) from businesses where (postal_code = '94103') AND (lower(address) LIKE '%market s%')

 * sqlite:///data/sfscores.sqlite
Done.


count(business_id)
110


Howmany businesses have "market" in their address and are **not** in postcode 94103?

In [36]:
%sql select count(business_id) from businesses where (postal_code <> '94103') AND (lower(address) LIKE '%market s%')

 * sqlite:///data/sfscores.sqlite
Done.


count(business_id)
179


How many businesses do **not** have "market" in their address and **are** in postcode 94103?

In [37]:
%sql select count(business_id) from businesses where (lower(address) NOT LIKE '%market s%') AND (postal_code = '91403')

 * sqlite:///data/sfscores.sqlite
Done.


count(business_id)
0


In [34]:
%sql select count(*) from violations

 * sqlite:///data/sfscores.sqlite
Done.


count(*)
40735


Join:

In [38]:
%%sql results_join <<
SELECT
    v.business_id, risk_category
FROM
    violations as v INNER JOIN businesses as b
ON
    v.business_id = b.business_id 
WHERE (postal_code = '94103')
  
ORDER BY
    v.business_id DESC


 * sqlite:///data/sfscores.sqlite
Done.
Returning data to local variable results_join


In [39]:
df_join = results_join.DataFrame()
df_join.shape

(3503, 2)

Sub-query (compare with join above):

In [40]:
%%sql results_subquery <<
SELECT
    v.business_id, risk_category
FROM
    violations as v
WHERE
    v.business_id
IN 
(SELECT b.business_id from businesses as b
WHERE (postal_code = '94103'))
ORDER BY
    v.business_id DESC

 * sqlite:///data/sfscores.sqlite
Done.
Returning data to local variable results_subquery


In [41]:
df_subquery = results_subquery.DataFrame()
df_subquery.shape

(3495, 2)

In [42]:
df_both = pd.concat([df_join,df_subquery])
df_both.shape

(6998, 2)

In [43]:
df_diff = df_both.drop_duplicates(keep=False)
df_diff.shape

(0, 2)

In [44]:
df_join.head(3)

,business_id,risk_category
0,9727,Low Risk
1,9727,Moderate Risk
2,9727,Moderate Risk


In [45]:
print(df_join.business_id.nunique(), df_subquery.business_id.nunique(), "\n")
print(df_join.risk_category.nunique(), df_subquery.risk_category.nunique(), "\n")
print(df_join.shape, df_subquery.shape, "\n")


465 465 

3 3 

(3503, 2) (3495, 2) 



In [46]:
%%sql 
SELECT
    COUNT(DISTINCT(business_id || date || ViolationTypeID || risk_category || description)), COUNT(*)
FROM
    violations

 * sqlite:///data/sfscores.sqlite
Done.


COUNT(DISTINCT(business_id || date || ViolationTypeID || risk_category || description)),COUNT(*)
40679,40735


In [47]:
%%sql 
SELECT
    COUNT(DISTINCT(business_id || postal_code)), COUNT(*)
FROM
    businesses
WHERE postal_code = '94103'

 * sqlite:///data/sfscores.sqlite
Done.


COUNT(DISTINCT(business_id || postal_code)),COUNT(*)
657,658
